In this section the API selected will be used so that we load our ES db

we ll use docker for elastic search and fill the details in the .ENV file

- NYT-Times NewsWire API
- NYT-Books API
- GoodReads

// end points
// list of book categories : https://api.nytimes.com/svc/books/v3/lists/names.json
// number of reviewed book : https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json
// good reads : https://www.goodreads.com/book/review_counts.json
//

// need to establish the following :
//1. connection to the ES container
//
//2. loading the api request from the two api
//2. 1 for each API define loops and requirements


1. Connection to the container // Windows installation

1.Install Docker: Download and install Docker Desktop for Windows from the official Docker website. Follow the installation instructions specific to your Windows version.
2.Launch Docker: Once Docker is installed, launch Docker Desktop from the start menu or desktop shortcut. Make sure it's running before proceeding to the next steps.
3.Open a Command Prompt: Open the Command Prompt or PowerShell on your Windows machine.
4.Pull the Elasticsearch Docker Image: Use the following command to pull the official Elasticsearch Docker image from Docker Hub:



5.Create a Docker Container
docker

6.Verify Elasticsearch Installation: Open your web browser and navigate to http://localhost:9200.


5.1 sauvegarder le fichier YAML dans le dossier du projet 

version: "3.0"


services:
  elasticsearch:
    container_name: es-container
    image: docker.elastic.co/elasticsearch/elasticsearch:8.1.2
    environment:
      - xpack.security.enabled=false
      - "discovery.type=single-node"
    networks:
      - es-net
    ports:
      - 9200:9200

  kibana:
    container_name: kb-container
    image: docker.elastic.co/kibana/kibana:8.1.2
    environment:
      - ELASTICSEARCH_HOSTS=http://es-container:9200
    networks:
      - es-net
    depends_on:
      - elasticsearch
    ports:
      - 5601:5601

networks:
  es-net:
    driver: bridge


puis la command dans le prompt
docker-compose up -d

Once the connection is established we want to analyse in details the API and load them one by one without overloading the API from NYT or GoodReads

To interact with ES firstly you need to : pip install elasticsearch


In [3]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk


##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for yhr NYT - nyt-wire 
##########################################################################

es = Elasticsearch(hosts = "http://@localhost:9200")


index_name = 'nyt-newswire'
newswire_index_body = {  'settings': {
        'number_of_shards': 2,
        'number_of_replicas': 1
    },
    'mappings': {
        'properties': {
            'abstract': {'type': 'text'},
            'byline': {'type': 'text'},
            'created_date': {'type': 'date'},
            'des_facet': {'type': 'keyword'},
            'first_published_date': {'type': 'date'},
            'geo_facet': {'type': 'keyword'},
            'item_type': {'type': 'keyword'},
            'kicker': {'type': 'text'},
            'material_type_facet': {'type': 'keyword'},
            'multimedia': {
                'type': 'nested',
                'properties': {
                    'caption': {'type': 'text'},
                    'copyright': {'type': 'text'},
                    'format': {'type': 'keyword'},
                    'height': {'type': 'integer'},
                    'subtype': {'type': 'keyword'},
                    'type': {'type': 'keyword'},
                    'url': {'type': 'text'},
                    'width': {'type': 'integer'}
                }
            },
            'org_facet': {'type': 'keyword'},
            'per_facet': {'type': 'keyword'},
            'published_date': {'type': 'date'},
            'section': {'type': 'keyword'},
            'slug_name': {'type': 'keyword'},
            'source': {'type': 'text'},
            'subheadline': {'type': 'text'},
            'subsection': {'type': 'keyword'},
            'thumbnail_standard': {'type': 'text'},
            'title': {'type': 'text'},
            'updated_date': {'type': 'date'},
            'uri': {'type': 'keyword'},
            'url': {'type': 'text'}
        }
    }
}


 # Create the index
response = es.indices.create(index=index_name, body=newswire_index_body)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_908\2741550789.py:58: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.indices.create(index=index_name, body=newswire_index_body)


Index created successfully.


In [8]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv

##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for yhr NYT - Book API -- best sellers
##########################################################################

es = Elasticsearch(hosts = "http://@localhost:9200")

index_name = 'nyt-books-best-sellers'
BS_books_index_body = {
     'settings': {
        'number_of_shards': 2,
        'number_of_replicas': 1
    },
     'mappings': {
         'properties': {
        'title': {'type': 'text'},
        'description': {'type': 'text'},
        'contributor': {'type': 'text'},
        'author': {'type': 'text'},
        'contributor_note': {'type': 'text'},
        'price': {'type': 'float'},
        'age_group': {'type': 'keyword'},
        'publisher': {'type': 'text'},
        'isbns': {
            'type': 'nested',
            'properties': {
                'isbn10': {'type': 'keyword'},
                'isbn13': {'type': 'keyword'},
            }
        },
        'ranks_history': {
            'type': 'nested',
            'properties': {
                'primary_isbn10': {'type': 'keyword'},
                'primary_isbn13': {'type': 'keyword'},
                'rank': {'type': 'integer'},
                'list_name': {'type': 'text'},
                'display_name': {'type': 'text'},
                'published_date': {'type': 'date'},
                'bestsellers_date': {'type': 'date'},
                'weeks_on_list': {'type': 'integer'},
                'ranks_last_week': {'type': 'integer', 'null_value': None},
                'asterisk': {'type': 'integer'},
                'dagger': {'type': 'integer'},
            }
        },
        'reviews': {
            'type': 'nested',
            'properties': {
                'book_review_link': {'type': 'keyword'},
                'first_chapter_link': {'type': 'keyword'},
                'sunday_review_link': {'type': 'keyword'},
                'article_chapter_link': {'type': 'keyword'},
              }
            },
         }
        }
    }


 # Create the index
response = es.indices.create(index=index_name, body=BS_books_index_body)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_908\3439416945.py:71: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.indices.create(index=index_name, body=BS_books_index_body)


Index created successfully.


In [9]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv

##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for the NYT - MOVIES API --
##########################################################################

es = Elasticsearch(hosts="http://@localhost:9200")

index_name = 'nyt-movies'
BS_books_index_body = {
    'settings': {
        'number_of_shards': 2,
        'number_of_replicas': 1
    },
    "mappings": {
        "properties": {
            "byline": {"type": "text"},
            "critics_pick": {"type": "integer"},
            "date_updated": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "display_title": {"type": "text"},
            "headline": {"type": "text"},
            "link": {
                "properties": {
                    "suggested_link_text": {"type": "text"},
                    "type": {"type": "keyword"},
                    "url": {"type": "text"}
                }
            },
            "mpaa_rating": {"type": "keyword"},
            "multimedia": {
                "properties": {
                    "height": {"type": "integer"},
                    "src": {"type": "text"},
                    "type": {"type": "keyword"},
                    "width": {"type": "integer"}
                }
            },
            "opening_date": {"type": "date", "format": "yyyy-MM-dd"},
            "publication_date": {"type": "date", "format": "yyyy-MM-dd"},
            "summary_short": {"type": "text"}
        }
    }

}

# Create the index
response = es.indices.create(index=index_name, body=BS_books_index_body)

# Check the response

if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_908\706243207.py:52: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.indices.create(index=index_name, body=BS_books_index_body)


Index created successfully.


The following are the requests and loading in the Elastic search DB


In [ ]:

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import time
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")

########################################################
########################################################
# NEWSWIRE API
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

#Define the variables used in the API
#Get the list of sections needed to call the API Newswire
reqSections =  requests.get(f'https://api.nytimes.com/svc/news/v3/content/section-list.json?&api-key={api_key}')
sectionsList= [item['section'] for item in reqSections.json()['results']]
print(len(sectionsList))
print(sectionsList)

########################################################
########################################################
#REMOVE OR COMMENT AFTER TEST  for tests ONLY ###
########################################################
sectionsList = sectionsList[:1]
print(sectionsList)
########################################################
########################################################
########################################################



# Create a connection to Elasticsearch  
es = Elasticsearch(hosts="http://@localhost:9200")  
# Proper index on the contenerize ES Db 
index_name = 'nyt-newswire'

# iterate through the list 
for section in sectionsList:
     
    #Request the Api
    content = requests.get(f'https://api.nytimes.com/svc/news/v3/content/all/{section}.json?&api-key={api_key}')
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']

    # Prepare the documents for bulk indexing
    actions = []
    for doc in docs:
        action = {
            "_index": index_name,
            "_source": doc
        }
        actions.append(action)

    # Perform the bulk indexing
    response = bulk(es, actions)

    # Check the response
    if not response[1]:
        print(f'{section} saved successfully')
    else:
        print('Failed to save content.')
    

    ######################################################
    time.sleep(12.02) ##### TO MODIFY ACCORDING API ALLOWANCE
    #################### 5 requests max per minute ######
    ######################################################
  


In [ ]:
########################################################
########################################################
# NEWSWIRE API -TEST ALL REQUEST
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

from elasticsearch import Elasticsearch
from pprint import pprint

# Create a connection to Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")
index_name = 'nyt-newswire'

# Define the search query
search_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the search request
response = es.search(index=index_name, body=search_query)

# Process and print the search results
pprint(response['hits']['hits'])

In [8]:
########################################################
########################################################
# NYT BOOKS API endpoint: best-sellers/history.json
# The API sends back 20 for each query 
# It is possible to get over 35K books review
########################################################
########################################################


from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import time
import json
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")



#Define the variables used in the API
#Get the list of sections needed to call the API Newswire



#connect to ES container in local 
es = Elasticsearch(hosts = "http://@localhost:9200")
index_name = 'nyt-books-best-sellers'

######################################################################################
# CHANGE THESE VALUES IF YOU NEED TO RESTART LOADING
# OR WANT TO TEST THE PROCESS 
######################################################################################
offsetvalue=0;    ###### IMPORTANT- OFFSETTING VALUE FOR THE START  OF THE WHILE LOOP
maxAPICall= 2     ##### To modify up to 500 if you want to load up to your daily limit
######################################################################################

offsetFactor=20 
endpointHits = 35311
continueLoading= True
APICallDailyIndex = 0  ##### MAX VAL  500


# iterate through the list 
while(continueLoading):
     
    APICallDailyIndex +=1
    continueLoading = APICallDailyIndex<maxAPICall

    #Request the Api
    content = requests.get(f"https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?offset={offsetvalue}&api-key={api_key}")
    
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']
    
   # Prepare the documents for bulk indexing
    actions = []
    for doc in docs:
        action = {
            "_index": index_name,
            "_source": doc
        }
        actions.append(action)

    # Perform the bulk indexing
    response = bulk(es, actions)

    # Check the response
    if not response[1]:
        print(f'{APICallDailyIndex*offsetFactor}`/35311 books saved successfully')
    else:
        print('Failed to save content.')

  
    ######################################################
    time.sleep(12) ##### TO MODIFY ACCORDING API ALLOWANCE
    ######################################################

print('end of the API loop')
    


{   'copyright': 'Copyright (c) 2023 The New York Times Company.  All Rights '
                 'Reserved.',
    'num_results': 35311,
    'results': [   {   'age_group': '',
                       'author': 'Diana Gabaldon',
                       'contributor': 'by Diana Gabaldon',
                       'contributor_note': '',
                       'description': 'The author of the Outlander novels '
                                      'gives tips on writing sex scenes, '
                                      'drawing on examples from the books.',
                       'isbns': [   {   'isbn10': '0399178570',
                                        'isbn13': '9780399178573'}],
                       'price': '0.00',
                       'publisher': 'Dell',
                       'ranks_history': [   {   'asterisk': 0,
                                                'bestsellers_date': '2016-08-20',
                                                'dagger': 0,
                 

In [9]:
########################################################
########################################################
# NYT BOOKS API endpoint: best-sellers/history 
#-TEST ALL REQUEST
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

from elasticsearch import Elasticsearch
from pprint import pprint

# Create a connection to Elasticsearch  
#connect to ES container in local 
es = Elasticsearch(hosts = "http://@localhost:9200")
index_name = 'nyt-books-best-sellers'


# Define the search query
search_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the search request
response = es.search(index=index_name, body=search_query)

# Process and print the search results
pprint(response['hits']['hits'])

C:\Users\ep211.000\AppData\Local\Temp\ipykernel_22436\2339415461.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


[{'_id': 'ftNv1YgB6W29Qs4cdL9L',
  '_index': 'nyt-books-best-sellers',
  '_score': 1.0,
  '_source': {'age_group': '',
              'author': 'JRR Tolkien',
              'contributor': 'by J.R.R. Tolkien',
              'contributor_note': '',
              'description': 'The love of a mortal man for an immortal elf, '
                             'which figures in “The Silmarillion” and is part '
                             'of the back story of “Lord of the Rings.” Edited '
                             'by Christopher Tolkien.',
              'isbns': [{'isbn10': '1328791823', 'isbn13': '9781328791825'}],
              'price': '0.00',
              'publisher': 'Houghton Mifflin Harcourt',
              'ranks_history': [{'asterisk': 0,
                                 'bestsellers_date': '2017-06-10',
                                 'dagger': 0,
                                 'display_name': 'Hardcover Fiction',
                                 'list_name': 'Hardcover Fictio

In [ ]:
########################################################
########################################################
# NYT - MOVIE API 
# The API sends back 20 for each query 
# It is possible to get over 31300K books review
########################################################
########################################################

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import time
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")



# Create a connection to Elasticsearch  
#connect to ES container in local 
index_name = 'nyt-movies'
es = Elasticsearch(hosts = "http://@localhost:9200")


########################################################
maxQueries= 2 ##### TO DEFINE !!!! According to API daily limit
########################################################
queryIndex=0     
offsetFactor=20

#API ARGUMENTS#############################################
type = "all"
offsetValue=0 ##  Max value tested == 31300 

# iterate through the list 
while(queryIndex<maxQueries): 
     
    # Create a connection to Elasticsearch  
    #Request the ApI
    content = requests.get(f'https://api.nytimes.com/svc/movies/v2/reviews/{type}.json?offset={offsetvalue}&api-key={api_key}')
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']

# Prepare the documents for bulk indexing
    ### this code could be made as a function but for
    # the sake of the cell singularity we leave it that way 
    actions = []
    for doc in docs:
        action = {
            "_index": index_name,
            "_source": doc
        }
        actions.append(action)

    # Perform the bulk indexing
    response = bulk(es, actions)

    # Check the response
    if not response[1]:
        print('saved successfully')
    else:
        print('Failed to save content.')

  ################################################################
    #Define the logic to increment in a writer has more than 20 entries
 
    # has more pages 
    hasMore = res['has_more']

    if not hasMore:
        offsetValue=0 
    else:
        offsetvalue =+ offsetFactor
        
    ################################################################


    ######################################################
    time.sleep(1) ##### TO MODIFY ACCORDING API ALLOWANCE
    ######################################################
    


In [ ]:
########################################################
########################################################
# NYT BOOKS API endpoint: best-sellers/history 
#-TEST ALL REQUEST
# The API sends back 500 articles meta for each section
# It is not possible to get more than 25000 articles
########################################################
########################################################

from elasticsearch import Elasticsearch
from pprint import pprint


# Create a connection to Elasticsearch  
#connect to ES container in local 
index_name = 'nyt-movies'
es = Elasticsearch(hosts = "http://@localhost:9200")


# Define the search query
search_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the search request
response = es.search(index=index_name, body=search_query)

# Process and print the search results
pprint(response['hits']['hits'])